In [2]:
import numpy as np

# Load the .npy file
data = np.load("C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/pats.npy")

# Get the shape and data type of the array
print(f'Shape: {data.shape}')
print(f'Data Type: {data.dtype}')
# Otherwise, you might want to look at just the first few rows and columns

Shape: (6, 3, 28, 360000)
Data Type: float64


In [3]:
import numpy as np

# Load the .npy file
data = np.load("C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/pats.npy")

# Get the number of subjects from the shape of the array
num_subjects = data.shape[0]

# Define a mapping from old subject numbers to new subject numbers
subject_number_mapping = {
    0: '01', 1: '02', 2: '03', 3: '04', 4: '05',
    5: '06', 6: '07', 7: '08', 8: '09', 9: '10',
    10: '11', 11: '12', 12: '13', 13: '14', 14: '16',
    15: '18', 16: '19', 17: '20', 18: '21', 19: '22'
}

# Iterate through each subject and save their data to a new .npy file
for i in range(num_subjects):
    subject_data = data[i]
    #subject_number = subject_number_mapping[i]  # Get the new subject number from the mapping
    filename = f'sub{subject_number}.npy'
    np.save(filename, subject_data)
    print(f'Saved {filename}')

print("All subjects' data have been saved to individual .npy files.")

NameError: name 'subject_number' is not defined

In [5]:
import numpy as np

# Load the .npy file
data = np.load("C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/pats.npy")

# Get the number of subjects from the shape of the array
num_subjects = data.shape[0]

# Start with patient 01
pat_num = 1

# Iterate through each subject and save their data to a new .npy file
for i in range(num_subjects):
    # Skip patient 03
    if pat_num == 3:
        pat_num += 1

    subject_data = data[i]
    filename = f'pat_{pat_num:02}.npy'  # Format the patient number with two digits
    np.save(filename, subject_data)
    print(f'Saved {filename}')

    # Increment patient number
    pat_num += 1

    # Stop if we reach patient 07
    if pat_num > 7:
        break

print("All subjects' data have been saved to individual .npy files.")


Saved pat_01.npy
Saved pat_02.npy
Saved pat_04.npy
Saved pat_05.npy
Saved pat_06.npy
Saved pat_07.npy
All subjects' data have been saved to individual .npy files.


In [9]:
data = np.load("sub01.npy")
print(f'Shape: {data.shape}')

Shape: (3, 28, 360000)


Creating individual csv files annotated with the timestamps for all frames

In [4]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta

def getMarkerOrder():
    return np.array([
        'LPPI', 'RPPI', 'LPPS', 'RPPS', 'LThiUpLat', 'LThiUpMed', 'LThiLoLat',
        'LThiLoMed', 'LShaUpLat', 'LShaUpMed', 'LShaLoLat', 'LShaLoMed',
        'RThiUpLat', 'RThiUpMed', 'RThiLoLat', 'RThiLoMed', 'RShaUpLat',
        'RShaUpMed', 'RShaLoLat', 'RShaLoMed', 'LHeePoDis', 'LHeePoPro',
        'LHeeLaTer', 'LHeeMeDia', 'RHeePoDis', 'RHeePoPro', 'RHeeLaTer',
        'RHeeMeDia'
    ])

# Define the directories
log_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/LogFiles'
npy_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/Dokumente/GitHub/PA-Exercube-ZHAW'
output_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate through all the .log files in the LogFiles directory
for log_file in os.listdir(log_files_dir):
    if log_file.startswith('sub') and log_file.endswith('_log_file.log'):
        # Extract the subject number from the log file name
        subject_number = log_file[3:5]
        # Construct the path to the corresponding .npy file
        npy_file_path = os.path.join(npy_files_dir, f'sub{subject_number}.npy')

        # Check if the corresponding .npy file exists
        if os.path.exists(npy_file_path):
            # Load the .npy file
            data = np.load(npy_file_path)

            # Read the log file and extract the timestamp
            with open(os.path.join(log_files_dir, log_file), 'r') as f:
                log_content = f.read()
            try:
                timestamp_str = log_content.split('Player crossed finish line! Race ends...')[0].rsplit('[', 1)[-1].split(']')[0]
                # Convert the timestamp string to datetime object and remove the timezone
                timestamp = pd.to_datetime(timestamp_str).tz_localize(None)
            except IndexError:
                print(f'Warning: Timestamp not found in {log_file}')
                continue  # Skip to the next iteration if the timestamp is not found

            # Subtract 25 minutes and add 1 second to get the start time
            start_timestamp = timestamp - timedelta(minutes=25, seconds=-1)

            # Create an array of timestamps for each frame
            num_frames = data.shape[2]
            seconds = np.arange(num_frames // 240)  # Each second has 240 frames
            timestamp_array = np.array([start_timestamp + timedelta(seconds=int(s)) for s in seconds for _ in range(240)])

            # Get the marker order
            marker_order = getMarkerOrder()

            # Create column names based on the marker order
            columns = [f'{marker}_Axis{axis}' for marker in marker_order for axis in ['X', 'Y', 'Z']]

            # Reshape the data array to 2D (flattening the axis and marker dimensions)
            data_reshaped = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

            # Transpose the reshaped data array so that each column represents a frame
            data_transposed = data_reshaped.T

            # Create a DataFrame
            df = pd.DataFrame(data_transposed, columns=columns)

            # Add the timestamps as a new column
            df['Timestamp'] = timestamp_array

            # Save the DataFrame to a new .csv file
            output_file_path = os.path.join(output_dir, f'sub{subject_number}_data.csv')
            df.to_csv(output_file_path, index=False)
            print(f'Saved {output_file_path}.')
        else:
            print(f'Warning: No corresponding .npy file found for {log_file}')

print('Finished processing all .npy and .log files.')


Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub01_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub02_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub03_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub04_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub05_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub06_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub07_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub08_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub09_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub10_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub11_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub12_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Pro

In [10]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta

def getMarkerOrder():
    return np.array([
        'LPPI', 'RPPI', 'LPPS', 'RPPS', 'LThiUpLat', 'LThiUpMed', 'LThiLoLat',
        'LThiLoMed', 'LShaUpLat', 'LShaUpMed', 'LShaLoLat', 'LShaLoMed',
        'RThiUpLat', 'RThiUpMed', 'RThiLoLat', 'RThiLoMed', 'RShaUpLat',
        'RShaUpMed', 'RShaLoLat', 'RShaLoMed', 'LHeePoDis', 'LHeePoPro',
        'LHeeLaTer', 'LHeeMeDia', 'RHeePoDis', 'RHeePoPro', 'RHeeLaTer',
        'RHeeMeDia'
    ])

# Define the directories
log_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/LogFiles'
npy_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/Dokumente/GitHub/PA-Exercube-ZHAW'
output_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Iterate through all the .log files in the LogFiles directory
for log_file in os.listdir(log_files_dir):
    if log_file.startswith('pat') and log_file.endswith('_log_file.log'):
        # Extract the patient number from the log file name
        patient_number = log_file[3:5]
        # Construct the path to the corresponding .npy file
        npy_file_path = os.path.join(npy_files_dir, f'pat_{patient_number}.npy')

        # Check if the corresponding .npy file exists
        if os.path.exists(npy_file_path):
            # Load the .npy file
            data = np.load(npy_file_path)

            # Read the log file and extract the timestamp
            with open(os.path.join(log_files_dir, log_file), 'r') as f:
                log_content = f.read()
            try:
                timestamp_str = log_content.split('Player crossed finish line! Race ends...')[0].rsplit('[', 1)[-1].split(']')[0]
                # Convert the timestamp string to datetime object and remove the timezone
                timestamp = pd.to_datetime(timestamp_str).tz_localize(None)
            except IndexError:
                print(f'Warning: Timestamp not found in {log_file}')
                continue  # Skip to the next iteration if the timestamp is not found

            # Subtract 25 minutes and add 1 second to get the start time
            start_timestamp = timestamp - timedelta(minutes=25, seconds=-1)

            # Create an array of timestamps for each frame
            num_frames = data.shape[2]
            seconds = np.arange(num_frames // 240)  # Each second has 240 frames
            timestamp_array = np.array([start_timestamp + timedelta(seconds=int(s)) for s in seconds for _ in range(240)])

            # Get the marker order
            marker_order = getMarkerOrder()

            # Create column names based on the marker order
            columns = [f'{marker}_Axis{axis}' for marker in marker_order for axis in ['X', 'Y', 'Z']]

            # Reshape the data array to 2D (flattening the axis and marker dimensions)
            data_reshaped = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))

            # Transpose the reshaped data array so that each column represents a frame
            data_transposed = data_reshaped.T

            # Create a DataFrame
            df = pd.DataFrame(data_transposed, columns=columns)

            # Add the timestamps as a new column
            df['Timestamp'] = timestamp_array

            # Save the DataFrame to a new .csv file
            output_file_path = os.path.join(output_dir, f'pat{patient_number}_data.csv')
            df.to_csv(output_file_path, index=False)
            print(f'Saved {output_file_path}.')
        else:
            print(f'Warning: No corresponding .npy file found for {log_file}')

print('Finished processing all .npy and .log files.')


Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat01_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat02_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat04_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat05_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat06_data.csv.
Saved C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat07_data.csv.
Finished processing all .npy and .log files.


Checking whether the timestamp creation worked as intended


In [5]:
import pandas as pd
import os

# Load the DataFrame
df = pd.read_csv("C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/sub01_data.csv")

# Get the first and last five entries in the dataframe
first_five_entries = df['Timestamp'].head(5)
last_five_entries = df['Timestamp'].tail(5)

# Get the unique timestamps, then take the first and last five of these
unique_timestamps = df['Timestamp'].unique()
first_five_unique = unique_timestamps[:5]
last_five_unique = unique_timestamps[-5:]
timestamp_1500 = df['Timestamp'].iloc[1499]

# Print the results
print("First five entries:")
print(first_five_entries)
print("\nLast five entries:")
print(last_five_entries)
print("\nFirst five unique timestamps:")
print(first_five_unique)
print("\nLast five unique timestamps:")
print(last_five_unique)
print(len(unique_timestamps))

First five entries:
0    2021-11-16 11:06:46
1    2021-11-16 11:06:46
2    2021-11-16 11:06:46
3    2021-11-16 11:06:46
4    2021-11-16 11:06:46
Name: Timestamp, dtype: object

Last five entries:
359995    2021-11-16 11:31:45
359996    2021-11-16 11:31:45
359997    2021-11-16 11:31:45
359998    2021-11-16 11:31:45
359999    2021-11-16 11:31:45
Name: Timestamp, dtype: object

First five unique timestamps:
['2021-11-16 11:06:46' '2021-11-16 11:06:47' '2021-11-16 11:06:48'
 '2021-11-16 11:06:49' '2021-11-16 11:06:50']

Last five unique timestamps:
['2021-11-16 11:31:41' '2021-11-16 11:31:42' '2021-11-16 11:31:43'
 '2021-11-16 11:31:44' '2021-11-16 11:31:45']
1500


Adding the exercises to the dataframe

In [6]:
import os
import pandas as pd
from datetime import datetime

# Define the directories
log_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/LogFiles'
csv_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'
output_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'

# Function to parse the timestamp and remove the timezone
def parse_timestamp(timestr):
    return datetime.strptime(timestr.split('+')[0].strip(), '%Y-%m-%d %H:%M:%S')

# Iterate through all the log files in the directory
for log_file in os.listdir(log_files_dir):
    if log_file.endswith('.log'):
        # Extract the subject number from the log file name
        subject_number = log_file.split('_')[0]

        # Paths for the corresponding CSV file
        csv_file_path = os.path.join(csv_files_dir, f'{subject_number}_data.csv')

        # Check if the corresponding CSV file exists
        if os.path.exists(csv_file_path):
            exercise_mapping = {}

            with open(os.path.join(log_files_dir, log_file), 'r', encoding='utf-8') as file:
                for line in file:
                    if "Exercise correct:" in line:
                        parts = line.split(']')
                        timestamp_str = parts[0].strip('[')
                        exercise_name = parts[1].split("'")[1]  # Extracting the exercise name
                        timestamp = parse_timestamp(timestamp_str)
                        exercise_mapping[timestamp] = exercise_name

            # Load the CSV data
            csv_data = pd.read_csv(csv_file_path)
            csv_data['Timestamp'] = pd.to_datetime(csv_data['Timestamp'])

            # Add an 'Exercise' column if it doesn't exist
            if 'Exercise' not in csv_data.columns:
                csv_data['Exercise'] = None

            for exercise_timestamp, exercise in exercise_mapping.items():
                # Find the rows with the matching timestamp and annotate them
                matching_indices = csv_data[csv_data['Timestamp'] == exercise_timestamp].index
                if not matching_indices.empty:
                    csv_data.loc[matching_indices, 'Exercise'] = exercise

            # Save the updated data
            updated_csv_file_path = os.path.join(output_dir, f'{subject_number}_updated_data.csv')
            csv_data.to_csv(updated_csv_file_path, index=False)
            print(f'Updated file for subject {subject_number} saved: {updated_csv_file_path}')
        else:
            print(f"CSV file not found for subject {subject_number}: {csv_file_path}")

print('Finished processing all log and CSV files.')

CSV file not found for subject pat01: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat01_data.csv
CSV file not found for subject pat02: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat02_data.csv
CSV file not found for subject pat04: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat04_data.csv
CSV file not found for subject pat05: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat05_data.csv
CSV file not found for subject pat06: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat06_data.csv
CSV file not found for subject pat07: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat07_data.csv
Updated file for subject sub01 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub01_updated_data.csv
Updated file for subject sub02 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub02_updated_data.csv
Updated file for subject sub03 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\sub03_upd

In [11]:
import os
import pandas as pd
from datetime import datetime

# Define the directories
log_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/LogFiles'
csv_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'
output_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'

# Function to parse the timestamp and remove the timezone
def parse_timestamp(timestr):
    return datetime.strptime(timestr.split('+')[0].strip(), '%Y-%m-%d %H:%M:%S')

# Iterate through all the log files in the directory
for log_file in os.listdir(log_files_dir):
    if log_file.startswith('pat') and log_file.endswith('_log_file.log'):
        # Extract the patient number from the log file name
        patient_number = log_file.split('_')[0]

        # Paths for the corresponding CSV file
        csv_file_path = os.path.join(csv_files_dir, f'{patient_number}_data.csv')

        # Check if the corresponding CSV file exists
        if os.path.exists(csv_file_path):
            exercise_mapping = {}

            with open(os.path.join(log_files_dir, log_file), 'r', encoding='utf-8') as file:
                for line in file:
                    if "Exercise correct:" in line:
                        parts = line.split(']')
                        timestamp_str = parts[0].strip('[')
                        exercise_name = parts[1].split("'")[1]  # Extracting the exercise name
                        timestamp = parse_timestamp(timestamp_str)
                        exercise_mapping[timestamp] = exercise_name

            # Load the CSV data
            csv_data = pd.read_csv(csv_file_path)
            csv_data['Timestamp'] = pd.to_datetime(csv_data['Timestamp'])

            # Add an 'Exercise' column if it doesn't exist
            if 'Exercise' not in csv_data.columns:
                csv_data['Exercise'] = None

            for exercise_timestamp, exercise in exercise_mapping.items():
                # Find the rows with the matching timestamp and annotate them
                matching_indices = csv_data[csv_data['Timestamp'] == exercise_timestamp].index
                if not matching_indices.empty:
                    csv_data.loc[matching_indices, 'Exercise'] = exercise

            # Save the updated data
            updated_csv_file_path = os.path.join(output_dir, f'{patient_number}_updated_data.csv')
            csv_data.to_csv(updated_csv_file_path, index=False)
            print(f'Updated file for patient {patient_number} saved: {updated_csv_file_path}')
        else:
            print(f"CSV file not found for patient {patient_number}: {csv_file_path}")

print('Finished processing all log and CSV files.')


Updated file for patient pat01 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat01_updated_data.csv
Updated file for patient pat02 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat02_updated_data.csv
Updated file for patient pat04 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat04_updated_data.csv
Updated file for patient pat05 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat05_updated_data.csv
Updated file for patient pat06 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat06_updated_data.csv
Updated file for patient pat07 saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\pat07_updated_data.csv
Finished processing all log and CSV files.


Checking whether the Exercises have been added

In [12]:
import pandas as pd

# Replace this with the path to your CSV file
csv_file_path = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/pat01_updated_data.csv'

# Load the CSV data into a DataFrame
df = pd.read_csv(csv_file_path)

# Assuming the exercises are listed in a column named 'Exercise'
# We count the number of non-NaN (not empty) entries in the 'Exercise' column
number_of_exercises = df['Exercise'].notna().sum()

print(f'Total number of exercises added: {number_of_exercises}')

Total number of exercises added: 124800


In [15]:
import pandas as pd

# Replace this with the path to your CSV file
csv_file_path = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/sub02_updated_data.csv'
# Load the CSV data into a DataFrame
df = pd.read_csv(csv_file_path)

# Get the list of all column names
column_names = df.columns.tolist()

# Print the list of column names
print("Column names in the CSV file:")
print(column_names)


Column names in the CSV file:
['LPPI_AxisX', 'LPPI_AxisY', 'LPPI_AxisZ', 'RPPI_AxisX', 'RPPI_AxisY', 'RPPI_AxisZ', 'LPPS_AxisX', 'LPPS_AxisY', 'LPPS_AxisZ', 'RPPS_AxisX', 'RPPS_AxisY', 'RPPS_AxisZ', 'LThiUpLat_AxisX', 'LThiUpLat_AxisY', 'LThiUpLat_AxisZ', 'LThiUpMed_AxisX', 'LThiUpMed_AxisY', 'LThiUpMed_AxisZ', 'LThiLoLat_AxisX', 'LThiLoLat_AxisY', 'LThiLoLat_AxisZ', 'LThiLoMed_AxisX', 'LThiLoMed_AxisY', 'LThiLoMed_AxisZ', 'LShaUpLat_AxisX', 'LShaUpLat_AxisY', 'LShaUpLat_AxisZ', 'LShaUpMed_AxisX', 'LShaUpMed_AxisY', 'LShaUpMed_AxisZ', 'LShaLoLat_AxisX', 'LShaLoLat_AxisY', 'LShaLoLat_AxisZ', 'LShaLoMed_AxisX', 'LShaLoMed_AxisY', 'LShaLoMed_AxisZ', 'RThiUpLat_AxisX', 'RThiUpLat_AxisY', 'RThiUpLat_AxisZ', 'RThiUpMed_AxisX', 'RThiUpMed_AxisY', 'RThiUpMed_AxisZ', 'RThiLoLat_AxisX', 'RThiLoLat_AxisY', 'RThiLoLat_AxisZ', 'RThiLoMed_AxisX', 'RThiLoMed_AxisY', 'RThiLoMed_AxisZ', 'RShaUpLat_AxisX', 'RShaUpLat_AxisY', 'RShaUpLat_AxisZ', 'RShaUpMed_AxisX', 'RShaUpMed_AxisY', 'RShaUpMed_AxisZ', 'RS

Removing rows where no exercises were performed and combining all csv files into a big file

In [16]:
import os
import pandas as pd

# Define the directories
csv_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'
output_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'
combined_csv_file_path = os.path.join(output_dir, 'combined_data.csv')

# List to hold data from all files
all_data = []

# Iterate through all CSV files in the directory that end with 'updated_data.csv'
for csv_file in os.listdir(csv_files_dir):
    if csv_file.endswith('updated_data.csv'):
        file_path = os.path.join(csv_files_dir, csv_file)
        subject_number = csv_file.split('_')[0]  # Extract subject number from filename

        # Read the CSV file
        df = pd.read_csv(file_path)

        # Remove rows where 'Exercise' column is empty
        df = df.dropna(subset=['Exercise'])

        # Add subject number as a new column
        df['Subject'] = subject_number

        # Append the processed DataFrame to the list
        all_data.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(combined_csv_file_path, index=False)
print(f'Combined CSV file saved: {combined_csv_file_path}')


Combined CSV file saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\combined_data.csv


In [13]:
import os
import pandas as pd

# Define the directories
csv_files_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'
output_dir = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit'
combined_csv_file_path = os.path.join(output_dir, 'combined_data_new.csv')  # New file name to avoid overwriting

# List to hold data from all files
all_data = []

# Iterate through all CSV files in the directory that start with 'pat' and end with 'updated_data.csv'
for csv_file in os.listdir(csv_files_dir):
    if csv_file.startswith('pat') and csv_file.endswith('updated_data.csv'):
        file_path = os.path.join(csv_files_dir, csv_file)
        subject_number = csv_file.split('_')[0]  # Extract subject number from filename

        # Read the CSV file
        df = pd.read_csv(file_path)

        # Remove rows where 'Exercise' column is empty
        df = df.dropna(subset=['Exercise'])

        # Add subject number as a new column
        df['Subject'] = subject_number

        # Append the processed DataFrame to the list
        all_data.append(df)

# Combine all data into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv(combined_csv_file_path, index=False)
print(f'Combined CSV file saved: {combined_csv_file_path}')


Combined CSV file saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit\combined_data_new.csv


In [1]:
import pandas as pd

# Define the path to the combined CSV file
combined_csv_file_path = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/combined_data.csv'

# Define the path for the subset CSV file
subset_csv_file_path = 'C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/combined_data_subset.csv'

# Load the combined CSV file
df = pd.read_csv(combined_csv_file_path)

# Create a subset of the data (e.g., first 300 rows)
subset_df = df.head(50)

# Save the subset to a new CSV file
subset_df.to_csv(subset_csv_file_path, index=False)

print(f'Subset CSV file saved: {subset_csv_file_path}')


Subset CSV file saved: C:/Users/nevio/OneDrive - ZHAW/5. Semester/Projektarbeit/combined_data_subset.csv


In [18]:
print(df.head(10))


   LPPI_AxisX  LPPI_AxisY  LPPI_AxisZ  RPPI_AxisX  RPPI_AxisY  RPPI_AxisZ  \
0  104.522430   32.492393  102.742126   32.939053  276.874207  279.375397   
1  104.420219   32.387367  102.616280   32.826744  276.805817  279.185791   
2  104.314377   32.294186  102.504173   32.720028  276.713287  279.006500   
3  104.209587   32.190201  102.399689   32.606133  276.575317  278.810852   
4  104.103706   32.091942  102.281235   32.498119  276.495026  278.600128   
5  103.996223   31.991394  102.169952   32.383110  276.367462  278.399963   
6  103.887329   31.880138  102.058784   32.261818  276.112213  278.215149   
7  103.777580   31.776316  101.937325   32.127773  276.025757  277.945282   
8  103.674828   31.666050  101.815521   32.008263  275.905518  277.729553   
9  103.568069   31.551319  101.684853   31.890947  275.717194  277.599243   

   LPPS_AxisX  LPPS_AxisY  LPPS_AxisZ  RPPS_AxisX  ...  RHeePoPro_AxisZ  \
0  263.721069  272.345917  260.211853  289.340607  ...        66.092735   
1 

test